# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 11, 12, 10

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 60


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [01:46<00:00,  1.78s/it]


# Check local statistics

In [2]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
15,PriceChange_LinearReg,6,6,0,80.00,80.00,80.00,100.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,40.00,40.00,60.00,60.00,60.00,60.00,60.00,60.00,40.00,60.00,40.00,-1.39,-1.13,-5.98,-3.04,-2.23,-5.05,-4.93,-5.51,-5.08,-4.18,-2.05,-1.89,-1.13,2.25,2.91,-1.46,-1.72,-0.66,-0.53,-0.66,-0.26,0.00,-0.26,0.00,5,68.333333,-1.832500,-8.333333,-9.162500
12,PriceChange_LinearReg,5,6,0,80.00,80.00,80.00,100.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,40.00,40.00,60.00,60.00,60.00,60.00,60.00,60.00,40.00,60.00,40.00,-1.39,-1.13,-5.98,-3.04,-2.23,-5.05,-4.93,-5.51,-5.08,-4.18,-2.05,-1.89,-1.13,2.25,2.91,-1.46,-1.72,-0.66,-0.53,-0.66,-0.26,0.00,-0.26,0.00,5,68.333333,-1.832500,-8.333333,-9.162500
16,PriceChange_LinearReg,6,8,0,87.50,87.50,87.50,100.00,87.50,87.50,62.50,62.50,62.50,62.50,62.50,62.50,62.50,50.00,50.00,62.50,62.50,62.50,62.50,62.50,62.50,50.00,62.50,50.00,-2.44,-2.44,-2.09,-1.61,-2.42,-2.40,-2.80,-3.08,-2.16,-3.02,-1.30,-1.63,-3.10,-0.73,-0.18,-2.22,-1.66,-1.06,-1.78,-3.15,-2.80,-2.26,-2.42,-2.26,8,67.187500,-2.125417,-22.500000,-17.003333
9,PriceChange_LinearReg,4,6,0,66.67,66.67,66.67,100.00,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,33.33,33.33,66.67,66.67,66.67,66.67,66.67,66.67,33.33,66.67,33.33,-1.39,-0.21,-1.69,-1.13,-1.34,-2.19,-2.97,-5.51,-5.08,-3.95,-2.05,-0.92,-1.13,2.25,2.91,-1.46,-1.72,-0.66,-0.53,-0.66,-0.26,0.00,-0.26,0.00,3,62.502083,-1.247917,-22.493750,-3.743750
45,PriceChange_LinearReg,16,6,0,90.91,81.82,81.82,77.27,72.73,68.18,63.64,63.64,63.64,63.64,63.64,59.09,63.64,54.55,59.09,63.64,63.64,59.09,63.64,50.00,40.91,40.91,45.45,36.36,-1.27,-1.10,-1.94,-1.64,-1.82,-1.51,-2.34,-3.15,-3.56,-3.76,-3.11,-1.99,-2.10,-2.40,-1.73,-2.53,-1.42,-0.98,-0.76,-0.23,1.09,0.87,1.30,2.06,22,62.122500,-1.417500,-173.305000,-31.185000
13,PriceChange_LinearReg,5,8,0,85.71,85.71,85.71,100.00,85.71,85.71,57.14,42.86,57.14,57.14,57.14,57.14,57.14,42.86,42.86,57.14,57.14,57.14,57.14,57.14,57.14,42.86,57.14,42.86,-2.16,-1.43,-1.69,-1.48,-2.62,-2.19,-2.62,0.35,-2.06,-3.95,-2.05,-0.92,-1.13,2.25,2.91,-1.46,-1.72,-0.66,-0.53,-0.66,-0.26,0.00,-0.26,0.00,7,61.902917,-1.014167,-56.679583,-7.099167
42,PriceChange_LinearReg,15,6,0,90.48,80.95,80.95,76.19,71.43,66.67,61.90,61.90,61.90,61.90,61.90,57.14,61.90,52.38,57.14,61.90,61.90,57.14,61.90,52.38,42.86,42.86,47.62,38.10,-1.20,-1.07,-1.69,-1.13,-1.34,-0.82,-2.26,-2.65,-2.60,-3.65,-3.01,-1.89,-1.74,-2.02,-1.39,-2.37,-1.22,-0.66,-0.66,-0.66,1.72,1.21,0.68,2.03,21,61.307917,-1.182917,-182.533750,-24.841250
39,PriceChange_LinearReg,14,6,0,90.48,80.95,80.95,76.19,71.43,66.67,61.90,61.90,61.90,61.90,61.90,57.14,61.90,52.38,57.14,61.90,61.90,57.14,61.90,52.38,42.86,42.86,47.62,38.10,-1.20,-1.07,-1.69,-1.13,-1.34,-0.82,-2.26,-2.65,-2.60,-3.65,-3.01,-1.89,-1.74,-2.02,-1.39,-2.37,-1.22,-0.66,-0.66,-0.66,1.72,1.21,0.68,2.03,21,61.307917,-1.182917,-182.533750,-24.841250
51,PriceChange

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [6]}, 
              'LinearReg': {'timeperiod': [6], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
20,72.930243,-1.306944,-879.743681,-157.711944,3071
19,73.131424,-1.325521,-881.025174,-153.987604,3038
17,73.016458,-1.380833,-906.773576,-147.793403,2972
18,72.718576,-1.329062,-908.998681,-150.019097,3005
16,73.201181,-1.421007,-909.151806,-143.919722,2933
15,73.250174,-1.485451,-924.197604,-142.200174,2911
12,74.196285,-1.676250,-927.744236,-130.194201,2805
13,73.203299,-1.593021,-954.058576,-131.837604,2836
11,73.823993,-1.782292,-954.826875,-126.921111,2770
